+++
title = "Principal component analysis"
menu = "main"
weight = 1
toc = true
aliases = ["pca"]
+++

## Resources

- [*Principal component analysis*](https://personal.utdallas.edu/~herve/abdi-awPCA2010.pdf) by Hervé Abdi and Lynne J. Williams is excellent at explaining PCA interpretation. It also covers some extensions to PCA.
- [*A Tutorial on Principal Component Analysis*](https://arxiv.org/pdf/1404.1100.pdf) by Jonathon Shlens goes into more detail on the intuition behind PCA, while also discussing its applicability and limits.
- I learnt PCA from [these](https://www.math.univ-toulouse.fr/~xgendre/ens/m2se/DataMining.pdf) lecture notes from Xavier Gendre. He provides a comprehensive walkthrough using a dataset of Skyrim bows.

## Data

PCA assumes you have a dataframe consisting of numerical variables. This includes booleans and integers.

As an example, let's use a dataset describing the energy mix of each country in the world, in 2019. The dataset is normalized into percentages, which makes countries comparable with one another.

In [1]:
import pandas as pd
from sklearn.impute import SimpleImputer
import prince
df=pd.read_excel("code.xlsx",sheet_name='Data2021')
df.index=df["Country"]

d=df.drop(["id","Country","indexreg","pop_2021"],axis=1)
imputer = SimpleImputer()
d = imputer.fit_transform(d)
d = pd.DataFrame(d)
ind = df.columns.tolist()
del ind[:4]
nb = d.columns.tolist()
d.index=df["Country"]
my_dict = dict(zip(nb, ind))
d = d.rename(columns=my_dict)



In [2]:
d

,sdg1_wpc,sdg1_320pov,sdg2_undernsh,sdg2_stunting,sdg2_wasting,sdg2_obesity,sdg2_trophic,sdg2_crlyld,sdg2_snmi,sdg3_matmort,...,sdg16_detain,sdg16_prs,sdg16_u5reg,sdg16_cpi,sdg16_clabor,sdg16_rsf,sdg16_justice,sdg17_govex,sdg17_govrev,sdg17_statperf
Country,,,,,,,,,,,,,,,,,,,,,
Afghanistan,12.247081,22.721925,25.600000,38.20,5.1,5.5,2.190000,2.165000,0.680000,638.000000,...,31.287,4.45605,42.3,16.000000,21.390000,40.19,0.450000,4.298,13.039000,49.756000
Albania,0.050000,5.840000,3.900000,11.30,1.6,21.7,2.383000,4.840000,0.829000,15.000000,...,44.876,3.28500,98.4,35.000000,3.300000,30.59,0.557000,6.771,25.144000,75.383000
Algeria,0.330000,2.420000,2.500000,9.80,2.7,27.4,2.199000,1.759000,0.705000,112.000000,...,12.000,4.07800,99.6,33.000000,4.260000,47.26,0.581000,10.159,23.461827,55.149000
Andorra,12.247081,22.721925,1.170000,2.58,0.7,25.6,2.279256,3.609472,0.818306,161.065574,...,42.857,4.45605,100.0,43.061798,10.903779,23.32,0.564839,7.814,23.461827,62.991035
Angola,53.830000,75.690000,17.300000,37.60,4.9,8.2,2.131000,0.753000,0.935000,241.000000,...,47.985,3.36800,25.0,29.000000,18.724000,34.06,0.500000,2.866,20.261000,54.946000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Venezuela, RB",53.450000,78.020000,27.400000,13.40,4.1,25.6,2.313000,3.395000,1.011000,125.000000,...,62.999,1.67900,81.3,14.000000,10.903779,47.60,0.451000,3.815,23.461827,43.865000
Vietnam,0.770000,3.960000,6.700000,23.80,5.8,2.1,2.218000,5.685000,0.622000,43.000000,...,12.386,4.41500,96.1,39.000000,13.120000,78.46,0.500000,6.361,23.461827,65.959000
"Yemen, Rep.",12.247081,22.721925,45.400000,46.40,16.4,17.1,2.127000,0.759000,0.796000,164.000000,...,40.179,3.20000,30.7,16.000000,10.903779,62.35,0.564839,5.584,23.461827,36.771000


In [2]:
d1=d[["sdg2_undernsh","sdg2_stunting","sdg2_wasting"]]
d. columns

Index(['sdg1_wpc', 'sdg1_320pov', 'sdg2_undernsh', 'sdg2_stunting',
       'sdg2_wasting', 'sdg2_obesity', 'sdg2_trophic', 'sdg2_crlyld',
       'sdg2_snmi', 'sdg3_matmort', 'sdg3_neonat', 'sdg3_u5mort', 'sdg3_tb',
       'sdg3_hiv', 'sdg3_ncds', 'sdg3_pollmort', 'sdg3_traffic', 'sdg3_lifee',
       'sdg3_fertility', 'sdg3_births', 'sdg3_vac', 'sdg3_uhc', 'sdg3_swb',
       'sdg4_earlyedu', 'sdg4_primary', 'sdg4_second', 'sdg4_literacy',
       'sdg5_familypl', 'sdg5_edat', 'sdg5_lfpr', 'sdg5_parl', 'sdg6_water',
       'sdg6_sanita', 'sdg6_freshwat', 'sdg6_wastewat', 'sdg6_scarcew',
       'sdg7_elecac', 'sdg7_cleanfuel', 'sdg7_co2twh', 'sdg7_ren',
       'sdg8_adjgrowth', 'sdg8_slavery', 'sdg8_accounts', 'sdg8_unemp',
       'sdg8_rights', 'sdg8_impacc', 'sdg9_intuse', 'sdg9_mobuse', 'sdg9_lpi',
       'sdg9_uni', 'sdg9_articles', 'sdg9_rdex', 'sdg10_gini', 'sdg10_palma',
       'sdg11_slums', 'sdg11_pm25', 'sdg11_pipedwat', 'sdg11_transport',
       'sdg12_msw', 'sdg12_ewaste', 'sdg

## Fitting

The `PCA` estimator implements the `fit/transform` API from scikit-learn.

In [3]:
pca = prince.PCA(
    n_components=5,
    n_iter=3,
    rescale_with_mean=True,
    rescale_with_std=True,
    copy=True,
    check_input=True,
    engine='sklearn',
    random_state=42
)
pca = pca.fit(d)



The available parameters are:

- `n_components` — the number of components that are computed. You only need two if your intention is to visualize the two major components.
- `n_iter` — the number of iterations used for computing the SVD.
- `rescale_with_mean` — whether to substract each column's mean
- `rescale_with_std` — whether to divide each column by it's standard deviation
- `copy` — if `False` then the computations will be done inplace which can have possible side-effects on the input data
- `engine` — what SVD engine to use (should be one of `['fbpca', 'sklearn']`)
- `random_state` — controls the randomness of the SVD results.



## Eigenvalues

The importance of a principal component is indicated by the proportion of dataset inertia it explains. This is called the explained inertia, and is obtained by dividing the eigenvalues obtained with SVD by the total inertia.

The ideal situation is when a large share of inertia is explained by a low number of principal components.

In [4]:
pca.eigenvalues_summary



,eigenvalue,% of variance,% of variance (cumulative)
component,,,
0,31.839,35.38%,35.38%
1,5.832,6.48%,41.86%
2,4.124,4.58%,46.44%
3,3.719,4.13%,50.57%
4,2.989,3.32%,53.89%


In [5]:
pca.eigenvalues_


array([31.83945759,  5.83166688,  4.12437699,  3.71947689,  2.98855009])

In [6]:
pca.percentage_of_variance_


array([35.3771751 ,  6.47962987,  4.5826411 ,  4.1327521 ,  3.32061121])

In [7]:
pca.cumulative_percentage_of_variance_


array([35.3771751 , 41.85680497, 46.43944607, 50.57219817, 53.89280937])

Eigenvalues can also be visualized with a [scree plot](https://www.wikiwand.com/en/Scree_plot).

In [8]:
pca.scree_plot()


alt.Chart(...)

## Coordinates

The row principal coordinates can be obtained once the `PCA` has been fitted to the data.

In [9]:
pca.transform(d).head()


component,0,1,2,3,4
Country,,,,,
Afghanistan,8.711433,-0.767190,1.083981,-2.521617,0.121936
Albania,-1.505947,-1.640909,-2.154430,-0.879124,1.129654
Algeria,-1.154828,-2.348811,-1.451375,-1.810223,0.342785
Andorra,-2.677564,-0.361133,-0.006919,0.333285,0.620346
Angola,9.114353,2.199494,-0.432276,1.514948,-0.506616


The `transform` method is in fact an alias for `row_coordinates`:

In [10]:
pca.transform(d).equals(pca.row_coordinates(d))


True

This is transforming the original dataset into factor scores.

The column coordinates are obtained during the PCA training process.

In [41]:
pca.column_coordinates_.iloc[40:47] 


component,0,1,2,3,4
variable,,,,,
sdg8_adjgrowth,-0.424546,0.207003,0.068993,-0.235708,0.326630
sdg8_slavery,0.297985,-0.051719,0.079636,-0.168784,-0.012380
sdg8_accounts,-0.803459,0.226382,0.000530,0.080107,-0.059095
sdg8_unemp,0.134338,-0.277267,0.043819,0.438944,-0.116985
sdg8_rights,-0.542704,0.400753,-0.035762,0.125164,-0.044922
sdg8_impacc,-0.398192,0.190429,0.734408,0.188422,0.407031
sdg9_intuse,-0.886931,-0.053888,-0.015106,-0.041325,-0.101775


In [12]:
pca.column_coordinates_ .index


Index(['sdg1_wpc', 'sdg1_320pov', 'sdg2_undernsh', 'sdg2_stunting',
       'sdg2_wasting', 'sdg2_obesity', 'sdg2_trophic', 'sdg2_crlyld',
       'sdg2_snmi', 'sdg3_matmort', 'sdg3_neonat', 'sdg3_u5mort', 'sdg3_tb',
       'sdg3_hiv', 'sdg3_ncds', 'sdg3_pollmort', 'sdg3_traffic', 'sdg3_lifee',
       'sdg3_fertility', 'sdg3_births', 'sdg3_vac', 'sdg3_uhc', 'sdg3_swb',
       'sdg4_earlyedu', 'sdg4_primary', 'sdg4_second', 'sdg4_literacy',
       'sdg5_familypl', 'sdg5_edat', 'sdg5_lfpr', 'sdg5_parl', 'sdg6_water',
       'sdg6_sanita', 'sdg6_freshwat', 'sdg6_wastewat', 'sdg6_scarcew',
       'sdg7_elecac', 'sdg7_cleanfuel', 'sdg7_co2twh', 'sdg7_ren',
       'sdg8_adjgrowth', 'sdg8_slavery', 'sdg8_accounts', 'sdg8_unemp',
       'sdg8_rights', 'sdg8_impacc', 'sdg9_intuse', 'sdg9_mobuse', 'sdg9_lpi',
       'sdg9_uni', 'sdg9_articles', 'sdg9_rdex', 'sdg10_gini', 'sdg10_palma',
       'sdg11_slums', 'sdg11_pm25', 'sdg11_pipedwat', 'sdg11_transport',
       'sdg12_msw', 'sdg12_ewaste', 'sdg

## Visualization

The row and column coordinates be visualized together with a scatter chart.

In [13]:
import matplotlib.pyplot as plt

pca.plot(
    d,
    x_component=0,
    y_component=1,
    color_by="Country",
    show_rows=False,
    show_columns=True
)


alt.LayerChart(...)

## Contributions

In [14]:
pca.row_contributions_.head().style.format('{:.0%}')  


component,0,1,2,3,4
Country,,,,,
Afghanistan,1%,0%,0%,1%,0%
Albania,0%,0%,1%,0%,0%
Algeria,0%,0%,0%,0%,0%
Andorra,0%,0%,0%,0%,0%
Angola,1%,0%,0%,0%,0%


Observations with high contributions and different signs can be opposed to help interpret the component, because these observations represent the two endpoints of this component.

Column contributions are also available.

In [15]:
pca.column_contributions_.style.format('{:.0%}')


component,0,1,2,3,4
variable,,,,,
sdg1_wpc,2%,2%,0%,1%,1%
sdg1_320pov,2%,2%,0%,1%,0%
sdg2_undernsh,1%,0%,0%,0%,0%
sdg2_stunting,2%,0%,0%,0%,0%
sdg2_wasting,1%,0%,1%,2%,0%
sdg2_obesity,1%,2%,0%,1%,2%
sdg2_trophic,1%,0%,0%,1%,1%
sdg2_crlyld,1%,0%,1%,0%,0%
sdg2_snmi,0%,1%,2%,3%,1%


## Cosine similarities

In [16]:
pca.row_cosine_similarities(d).head()

component,0,1,2,3,4
Country,,,,,
Afghanistan,0.569709,0.004419,0.008821,0.047734,0.000112
Albania,0.042351,0.050282,0.086678,0.014433,0.023831
Algeria,0.034800,0.143959,0.054967,0.085508,0.003066
Andorra,0.231328,0.004208,0.000002,0.003584,0.012417
Angola,0.708197,0.041243,0.001593,0.019566,0.002188


In [19]:
pca.column_cosine_similarities_.loc["sdg1_wpc"]

component
0    0.525964
1    0.139625
2    0.008451
3    0.049335
4    0.025791
Name: sdg1_wpc, dtype: float64

## Column correlations

In [68]:
pca.column_correlations

component,0,1
variable,,
sdg1_wpc,0.710307,0.369959
sdg2_undernsh,0.675747,0.116016
sdg2_wasting,0.598569,-0.068580
sdg2_obesity,-0.452813,-0.334274
sdg2_trophic,-0.488513,0.074736
...,...,...
sdg17_govrev,-0.150003,-0.148826
sdg17_cohaven,-0.507715,0.405246
sdg17_statperf,-0.705123,0.236103


In [72]:
print(pca.column_correlations.loc[:, 1])


variable
sdg1_wpc          0.369959
sdg2_undernsh     0.116016
sdg2_wasting     -0.068580
sdg2_obesity     -0.334274
sdg2_trophic      0.074736
                    ...   
sdg17_govrev     -0.148826
sdg17_cohaven     0.405246
sdg17_statperf    0.236103
sdg1_320pov       0.302019
sdg2_stunting     0.059505
Name: 1, Length: 93, dtype: float64


In [37]:
(pca.column_correlations ** 2).equals(pca.column_cosine_similarities_)

True

## Inverse transformation

You can transform row projections back into their original space by using the `inverse_transform` method.

In [38]:
reconstructed = pca.inverse_transform(pca.transform(d))
reconstructed = reconstructed.rename(columns=my_dict)
reconstructed

,sdg1_wpc,sdg1_320pov,sdg2_undernsh,sdg2_stunting,sdg2_wasting,sdg2_obesity,sdg2_trophic,sdg2_crlyld,sdg2_snmi,sdg2_pestexp,...,sdg16_u5reg,sdg16_cpi,sdg16_clabor,sdg16_weaponsexp,sdg16_rsf,sdg16_justice,sdg17_govex,sdg17_govrev,sdg17_cohaven,sdg17_statperf
Country,,,,,,,,,,,,,,,,,,,,,
Afghanistan,31.637092,54.426534,20.448619,33.749914,8.721041,13.643488,2.140569,1.651223,0.974551,20.289849,...,59.936885,19.733400,21.669143,-0.322255,47.863996,0.469739,4.569260,21.789818,-7.167059,41.096851
Albania,2.118716,9.072299,6.796926,12.810639,3.204574,22.211958,2.300665,3.200180,0.766722,32.488566,...,94.180877,41.506758,7.251947,0.192740,35.836240,0.571416,8.853333,24.478060,8.380436,68.338766
Algeria,1.658282,9.093567,6.968813,13.427101,3.713668,23.267172,2.288164,3.381582,0.809104,31.671903,...,94.430384,39.071491,7.653642,0.113645,38.929297,0.562903,8.497506,24.878211,5.416143,64.730039
Andorra,5.456196,12.000947,6.023309,10.917500,3.185222,22.337295,2.319799,4.412624,0.790118,30.835090,...,92.453193,49.819080,7.471575,0.433067,32.652483,0.592887,8.938649,24.257293,22.600014,68.371076
Angola,39.566221,63.556436,22.270117,35.137937,7.942246,8.263248,2.158780,0.952289,0.867009,20.562874,...,52.190303,25.994700,23.013771,-0.088377,38.431780,0.485242,5.046715,19.801963,3.363412,48.849274
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Venezuela, RB",9.942967,22.807272,11.688518,21.028777,5.698509,20.872367,2.225527,2.565872,0.885224,27.943641,...,84.025347,28.447050,12.393354,-0.159949,45.029922,0.519929,6.949075,24.236650,-5.287205,54.522394
Vietnam,5.022343,13.872751,8.362035,15.362041,3.981096,21.653226,2.278237,3.051446,0.803258,30.943771,...,90.631964,37.984101,8.943747,0.100207,38.345944,0.556622,8.261605,24.336843,5.123576,64.120411
"Yemen, Rep.",27.472739,49.265199,19.305402,32.555255,8.823999,15.982784,2.138592,1.928153,1.007275,20.672417,...,64.140799,17.825680,20.645091,-0.401499,51.136569,0.466659,4.535186,22.646809,-11.123047,39.107118


This allows measuring the reconstruction error of the PCA. This is usually defined as the $L^2$ norm between the reconstructed dataset and the original dataset.

In [39]:
import numpy as np

np.linalg.norm(reconstructed.values - d.values, ord=2)

129636.91156983803

## Supplementary data

Active rows and columns make up the dataset you fit the PCA with. Anything you provide afterwards is considered supplementary data.

For example, we can fit the PCA on all countries that are not part of North America.

In [48]:
active = d.query("Country != 'Algeria'")
pca = prince.PCA().fit(active)

The data for North America can still be projected onto the principal components.

In [41]:
pca.transform(d).query("Country == 'Algeria'")

component,0,1,2
Country,,,
Algeria,-1.067475,-2.466682,-1.40029


As for supplementary, they must be provided during the `fit` call.

In [53]:
pca = prince.PCA().fit(d, supplementary_columns=([ 'sdg1_320pov', 'sdg2_stunting']))    
pca.column_correlations

component,0,1
variable,,
sdg1_wpc,0.710307,0.369959
sdg2_undernsh,0.675747,0.116016
sdg2_wasting,0.598569,-0.068580
sdg2_obesity,-0.452813,-0.334274
sdg2_trophic,-0.488513,0.074736
...,...,...
sdg17_govrev,-0.150003,-0.148826
sdg17_cohaven,-0.507715,0.405246
sdg17_statperf,-0.705123,0.236103


In [67]:
print(pca.column_correlations.loc[:, 1])


variable
sdg1_wpc          0.369959
sdg2_undernsh     0.116016
sdg2_wasting     -0.068580
sdg2_obesity     -0.334274
sdg2_trophic      0.074736
                    ...   
sdg17_govrev     -0.148826
sdg17_cohaven     0.405246
sdg17_statperf    0.236103
sdg1_320pov       0.302019
sdg2_stunting     0.059505
Name: 1, Length: 93, dtype: float64


There can be supplementary rows and columns at the same time.

In [49]:
pca = prince.PCA().fit(active, supplementary_columns=[ 'sdg1_320pov', 'sdg2_stunting'])

## Performance

Under the hood, Prince uses a [randomised version of SVD](https://research.facebook.com/blog/2014/9/fast-randomized-svd/). This is much faster than traditional SVD. However, the results may have a small inherent randomness. This can be controlled with the `random_state` parameter. The accurary of the results can be increased by providing a higher `n_iter` parameter.

By default `prince` uses scikit-learn's randomized SVD implementation -- the one used in [`TruncatedSVD`](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.TruncatedSVD.html).

Prince supports different SVD backends. For the while, the only other supported randomized backend is [Facebook's randomized SVD implementation](https://research.facebook.com/blog/fast-randomized-svd/), called [fbpca](http://fbpca.readthedocs.org/en/latest/):

In [28]:
prince.PCA(engine='fbpca')

PCA(engine='fbpca')

You can also use a non-randomized SVD implementation, using the [`scipy.linalg.svd` function](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.svd.html):

In [29]:
prince.PCA(engine='scipy')

PCA(engine='scipy')

Here is a very crude benchmark that compares each engine on different dataset sizes.

In [30]:
import itertools
import numpy as np
import pandas as pd

N = [100, 10_000, 100_000]
P = [3, 10, 30]
ENGINES = ['sklearn', 'fbpca', 'scipy']

perf = []

for n, p, engine in itertools.product(N, P, ENGINES):

    # Too slow
    if engine == 'scipy' and n > 10_000:
        continue

    X = pd.DataFrame(np.random.random(size=(n, p)))
    duration = %timeit -q -n 1 -r 3 -o prince.PCA(engine=engine, n_iter=3).fit(X)
    perf.append({
        'n': n,
        'p': p,
        'engine': engine,
        'seconds': duration.average
    })

In [31]:
(
    pd.DataFrame(perf)
    .set_index(['n', 'p'])
    .groupby(['n', 'p'], group_keys=False)
    .apply(lambda x: x[['engine', 'seconds']]
    .sort_values('seconds'))
)

engine   seconds
n      p                    
100    3     fbpca  0.003531
       3     scipy  0.004656
       3   sklearn  0.006733
       10    scipy  0.002478
       10    fbpca  0.002495
       10  sklearn  0.004621
       30    fbpca  0.002406
       30  sklearn  0.003195
       30    scipy  0.003921
10000  3     fbpca  0.003596
       3   sklearn  0.009349
       3     scipy  1.435681
       10    fbpca  0.009008
       10  sklearn  0.017285
       10    scipy  2.839416
       30    fbpca  0.022735
       30  sklearn  0.030653
       30    scipy  6.427675
100000 3     fbpca  0.023924
       3   sklearn  0.079227
       10    fbpca  0.117928
       10  sklearn  0.207227
       30    fbpca  0.217809
       30  sklearn  0.335177